In [97]:
%load_ext autoreload
%autoreload 2
import numpy as np
from PIL import Image, ImageOps
import os
from utils.H264 import *
from utils.ycbcr_conv import *
from utils.check import *
from pathlib import Path

# create directrot output if not exist
if not os.path.exists("output"):
    os.makedirs("output")

def read_image_into_rgb(path):
    img = Image.open(path)
    img = ImageOps.exif_transpose(img)
    img = img.convert("RGB")
    rgb = np.array(img)
    print(rgb.shape, rgb.dtype)              # (H, W, 3) uint8
    return rgb

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [98]:
# ipynb_root = Path(__file__).parent
input_img_path = Path("img") / "trump.jpg"
output_img_path = Path("output") / "recover_trump.jpg"
rgb_image = read_image_into_rgb(input_img_path)
ycbcr_image = rgb_2_ycbcr(rgb_image=rgb_image)
recover_rgb_image = ycbcr_2_rgb(ycbcr_image=ycbcr_image)
Image.fromarray(recover_rgb_image, mode="RGB").save(output_img_path, quality=90, optimize=True, progressive=True)

(512, 512, 3) uint8


In [99]:
original_image = read_image_into_rgb(input_img_path)
recover_image = read_image_into_rgb(output_img_path)
diff_two_tensor(A=original_image, B=recover_image)

(512, 512, 3) uint8
(512, 512, 3) uint8
convertion diff: 0.5409317016601562


In [100]:
Y_tensor = ycbcr_image[...,0]
print(Y_tensor.shape, Y_tensor.dtype)

(512, 512) uint8


In [101]:
frames, _ = macro_block_partition(Y_tensor=Y_tensor)

(16, 16, 32, 32) (512, 512)
convertion diff: 0.0


In [102]:
traget_frame = frames[1, 1]
op_mode = [0, 0, 0, 1]
QP = 13
blocks, _ = consume_a_frame(frame=traget_frame, op_mode=op_mode, QP=QP)

(2, 2, 16, 16) (32, 32)
convertion diff: 0.0
at blk (0, 0) we have op: 0
omg:  (16, 16) (16, 16) (16, 16)
haha:  (16, 16)
123456:  (16, 16)
float64


TypeError: ufunc 'right_shift' not supported for the input types, and the inputs could not be safely coerced to any supported types according to the casting rule ''safe''